Import necessary dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
from datetime import datetime

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# elif torch.backends.mps.is_available():
#     device = torch.device("mps:0")
else:
    device = torch.device("cpu")

Load data

In [4]:
data = pd.read_csv('IoT_Modbus.csv')

# TODO: Complete EDA

Combine 'date' and 'time' into a single datetime column

In [5]:
data['date'] = data['date'].str.strip()
data['time'] = data['time'].str.strip()

In [6]:
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format='%d-%b-%y %H:%M:%S')

Extract time features

In [7]:
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour
data['minute'] = data['datetime'].dt.minute
data['second'] = data['datetime'].dt.second
data['dayofweek'] = data['datetime'].dt.dayofweek

Time series models need to ensure that the data set is arranged in time order

In [8]:
# Sort the data by datetime
data = data.sort_values(by='datetime')

# Drop the original date, time, and timestamp columns
data.drop(['date', 'time', 'datetime', 'type'], axis=1, inplace=True)

# Adjust feature order
order = ['year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek', 'FC1_Read_Input_Register', 'FC2_Read_Discrete_Value', 'FC3_Read_Holding_Register', 'FC4_Read_Coil', 'label']
data = data[order].astype('int32')

# Split the dataset (Sequential Split)

In [9]:
# Calculate split points
split_idx = int(len(data) * 0.8)

# Split the data set, keeping order
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

train_data_copy = train_data.copy()

split_idx_train_validate = int(len(train_data_copy) * 0.8)

# Training data is the first 80%
train_data = train_data_copy.iloc[:split_idx_train_validate]
# Validation data is the last 20%
validation_data = train_data_copy.iloc[split_idx_train_validate:]

# Separate features and labels
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_validate = validation_data.drop('label', axis=1)
y_validate = validation_data['label']

X_test = test_data.drop('label', axis=1)
y_test = test_data['label']

In [10]:
X_train.columns

Index(['year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek',
       'FC1_Read_Input_Register', 'FC2_Read_Discrete_Value',
       'FC3_Read_Holding_Register', 'FC4_Read_Coil'],
      dtype='object')

# Data preprocessing (Normalization)

In [11]:
feature_columns = X_train.columns
scaler = MinMaxScaler()

X_train[feature_columns] = scaler.fit_transform(X_train[feature_columns]).astype('float32')
X_validate[feature_columns] = scaler.fit_transform(X_validate[feature_columns]).astype('float32')
X_test[feature_columns] = scaler.transform(X_test[feature_columns]).astype('float32')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183804 entries, 541 to 149866
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       183804 non-null  float32
 1   month                      183804 non-null  float32
 2   day                        183804 non-null  float32
 3   hour                       183804 non-null  float32
 4   minute                     183804 non-null  float32
 5   second                     183804 non-null  float32
 6   dayofweek                  183804 non-null  float32
 7   FC1_Read_Input_Register    183804 non-null  float32
 8   FC2_Read_Discrete_Value    183804 non-null  float32
 9   FC3_Read_Holding_Register  183804 non-null  float32
 10  FC4_Read_Coil              183804 non-null  float32
dtypes: float32(11)
memory usage: 9.1 MB


# Execution model
## Create model

In [12]:
class LightweightLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, width_multiplier=1.0):
        super(LightweightLSTM, self).__init__()
        # Adjust hidden size based on the width multiplier
        adjusted_hidden_size = int(hidden_size * width_multiplier)

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, adjusted_hidden_size, num_layers=num_layers, batch_first=True)

        self.linear_1 = nn.Linear(adjusted_hidden_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # LSTM layer
        lstm_out, _ = self.lstm(x)

        # Take the output of the last time step
        last_time_step_out = lstm_out[:, -1, :]

        # Output layer
        x = self.linear_1(last_time_step_out)
        out = self.linear_2(x)
        return out

Initialize model

In [13]:
features_num = X_train.shape[1]
hidden_neurons_num = 512
output_neurons_num = 1
lstm_num_layers = 2
multiplier = 0.5

model = LightweightLSTM(features_num, hidden_neurons_num, output_neurons_num, lstm_num_layers, multiplier).to(device)

In [14]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device=device)

Build loss functions and optimizers

In [15]:
weights = torch.tensor([1, class_weights[1]], dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(torch.FloatTensor([weights[1] / weights[0]])).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)

Construct Data Loader

In [16]:
batch_size = 128

# Use to_numpy is much better than .values
X_train_tensor: torch.Tensor = torch.tensor(X_train.to_numpy()).float().unsqueeze(1).to(device)
X_validate_tensor: torch.Tensor = torch.tensor(X_validate.to_numpy()).float().unsqueeze(1).to(device)

y_train_tensor: torch.Tensor = torch.tensor(y_train.to_numpy()).float().unsqueeze(1).to(device)
y_validate_tensor: torch.Tensor = torch.tensor(y_validate.to_numpy()).float().unsqueeze(1).to(device)


train_dataset: torch.utils.data.dataset.TensorDataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader: torch.utils.data.dataloader.DataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

validation_dataset: torch.utils.data.dataset.TensorDataset = TensorDataset(X_validate_tensor, y_validate_tensor)
validation_loader: torch.utils.data.dataloader.DataLoader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

Training model

In [17]:
num_epochs = 100
pbar = tqdm(total=num_epochs)
loss_list = [None] * num_epochs
acc_list = [None] * num_epochs

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:    
        # FP
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # BP and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate indicators
    model.eval()

    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in validation_loader:
            outputs = model(inputs)
            
            probabilities = torch.sigmoid(outputs)
            
            loss = criterion(probabilities, labels)
            
            # Calculate indicators
            running_loss += loss.item() * inputs.size(0)

            predictions = (probabilities > 0.5).float().to(device)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(validation_loader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    
    loss_list[epoch] = epoch_loss
    acc_list[epoch] = accuracy
    
    pbar.update(1)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss}, Accuracy: {accuracy}')

pbar.close()

  1%|          | 1/100 [00:11<18:18, 11.10s/it]

Epoch [1/100], Loss: 2.5925983849558847, Accuracy: 0.3014515462122696


  2%|▏         | 2/100 [00:21<17:09, 10.51s/it]

Epoch [2/100], Loss: 2.3723785992491186, Accuracy: 0.3014515462122696


  3%|▎         | 3/100 [00:31<16:52, 10.44s/it]

...

 12%|█▏        | 12/100 [02:05<15:29, 10.56s/it]

Epoch [12/100], Loss: 2.5999719525070506, Accuracy: 0.3014515462122696


 13%|█▎        | 13/100 [02:16<15:34, 10.74s/it]

Epoch [13/100], Loss: 2.6168696267055207, Accuracy: 0.3014515462122696


KeyboardInterrupt: 

Visualizing the training process

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_list, label='Training Loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Draw accuracy curve
plt.subplot(1, 2, 2)
plt.plot(acc_list, label='Training Accuracy')
plt.title('Training Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Unseen test set performance

In [ ]:
X_test_tensor = torch.tensor(X_test.values).float().unsqueeze(1).to(device)

model.eval()
outputs = model(X_test_tensor)
with torch.no_grad():
    probabilities = torch.sigmoid(outputs)
    predictions = (probabilities > 0.5).float().cpu().numpy()

    # Calculate indicators
    acc = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)

    print("Accuracy: ", acc, ", Precision: ", precision, ", Recall: ", recall, ", F1: ", f1)

Save model

In [ ]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_filename = f"model_{current_time}.pt"
torch.save(model.state_dict(), model_filename)

print("Model saved as:", model_filename)